# Descubra as melhores notas de matemática do ENEM 2016

Você deverá criar um modelo para prever a nota da prova de matemática de quem participou do ENEM 2016. Para isso, usará Python, Pandas, Sklearn e Regression.

### Detalhes

O contexto do desafio gira em torno dos resultados do ENEM 2016 (disponíveis no arquivo train.csv). Este arquivo, e apenas ele, deve ser utilizado para todos os desafios. Qualquer dúvida a respeito das colunas, consulte o [Dicionário dos Microdados do Enem 2016](https://s3-us-west-1.amazonaws.com/acceleration-assets-highway/data-science/dicionario-de-dados.zip).

No arquivo test.csv crie um modelo para prever nota da prova de matemática (coluna `NU_NOTA_MT`) de quem participou do ENEM 2016. 

Salve sua resposta em um arquivo chamado answer.csv com duas colunas: `NU_INSCRICAO` e `NU_NOTA_MT`.

In [1]:
#Importação de bibliotecas e pacotes
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import cross_validate

In [2]:
#Funções
def cria_df_aux(df):
    df_aux = pd.DataFrame({'Type': df.dtypes,
                          'Missing': df.isna().sum(),
                           'Size': df.shape[0],
                           'Unique': df.nunique()
                     })
    df_aux['Missing_%']= df_aux.Missing/df_aux.Size * 100
    df_aux
    return df_aux

In [3]:
#Importanção dos bancos de dados
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
#Inicial observação da base train
train.head(5)

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,...,5.0,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,...,NaN,A,A,C,A,B,A,A,C,A
2,3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,...,NaN,A,A,A,A,C,A,A,B,A
3,4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,...,5.0,C,A,A,A,A,D,A,A,A
4,5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,...,NaN,A,A,A,A,A,A,A,A,A


In [5]:
#Inicial observação da base test
test.head(5)

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,41,PR,22,F,3,1,1,5,1,...,40.0,420.0,B,A,C,A,A,C,C,A
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,21,MA,26,F,3,1,1,8,1,...,100.0,580.0,E,B,C,B,B,B,F,A
2,b38a03232f43b11c9d0788abaf060f7366053b6d,23,CE,21,M,1,1,2,0,2,...,80.0,320.0,E,E,D,B,B,A,NaN,A
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,15,PA,27,F,3,1,1,8,1,...,NaN,NaN,H,E,G,B,B,A,NaN,A
4,715494628a50142ce8cb17191cfe6d0f3cae0934,41,PR,18,M,1,1,2,0,2,...,0.0,320.0,D,H,H,C,B,A,NaN,A


## Criação de base de treino

Nota-se que a base de test possui 47 colunas apenas, contra 167 da base de treino. Desta forma, faz sentido treinar o modelo apenas com os dados que ele irá receber para a previsão. Vamos analisar as colunas que há em comum em ambos.

In [6]:
columns_train = train.columns
columns_test = test.columns

common_columns = list(set(columns_test).intersection(columns_train))
len(common_columns)

47

Nota-se que todos as colunas da base test encontram-se na base train. Vamos passar a analisar quais colunas são essas, analisando sua relevância para o problema proposto.

In [7]:
common_columns.sort()
print(common_columns)

['CO_PROVA_CH', 'CO_PROVA_CN', 'CO_PROVA_LC', 'CO_PROVA_MT', 'CO_UF_RESIDENCIA', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_DISCALCULIA', 'IN_DISLEXIA', 'IN_GESTANTE', 'IN_IDOSO', 'IN_SABATISTA', 'IN_SURDEZ', 'IN_TREINEIRO', 'NU_IDADE', 'NU_INSCRICAO', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q027', 'Q047', 'SG_UF_RESIDENCIA', 'TP_ANO_CONCLUIU', 'TP_COR_RACA', 'TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO', 'TP_ESCOLA', 'TP_LINGUA', 'TP_NACIONALIDADE', 'TP_PRESENCA_CH', 'TP_PRESENCA_CN', 'TP_PRESENCA_LC', 'TP_SEXO', 'TP_STATUS_REDACAO', 'TP_ST_CONCLUSAO']


### Observações para a seleção prévia de variáveis

Notadamente devemos remover as colunas 'NU_INSCRICAO' e 'SG_UF_RESIDENCIA', pois a primeira apenas indica o número de inscrição do aluno e a segunda é a mesma coluna que 'CO_UF_RESIDENCIA', sendo essa já codificada.

In [8]:
features = common_columns

In [9]:
features.remove('NU_INSCRICAO')
features.remove('SG_UF_RESIDENCIA')

Outra coluna que me parece desnecessária é a coluna TP_STATUS_REDACAO, pois me parece que caso sua indicação seja diferente de 1 (redação sem problemas) a nota da redação será 0. Vamos verificar:

In [10]:
status_redacao = train.loc[:, ['TP_STATUS_REDACAO', 'NU_NOTA_REDACAO']]
status_redacao

,TP_STATUS_REDACAO,NU_NOTA_REDACAO
0,1.0,520.0
1,1.0,580.0
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
13725,1.0,440.0
13726,1.0,560.0
13727,1.0,600.0
13728,1.0,460.0


In [11]:
status_redacao.query('TP_STATUS_REDACAO > 1 & NU_NOTA_REDACAO!="0"')

,TP_STATUS_REDACAO,NU_NOTA_REDACAO


Como ficou comprovado que caso o valor seja diferente de 1 em 'TP_STATUS_REDACAO' a nota da redação será 0, vamos remover também essa coluna.

In [12]:
features.remove('TP_STATUS_REDACAO')

In [13]:
#Dataframe auxiliar na análise
cria_df_aux(train[features])

,Type,Missing,Size,Unique,Missing_%
CO_PROVA_CH,object,0,13730,10,0.000000
CO_PROVA_CN,object,0,13730,10,0.000000
CO_PROVA_LC,object,0,13730,9,0.000000
CO_PROVA_MT,object,0,13730,9,0.000000
CO_UF_RESIDENCIA,int64,0,13730,27,0.000000
IN_BAIXA_VISAO,int64,0,13730,2,0.000000
IN_CEGUEIRA,int64,0,13730,1,0.000000
IN_DISCALCULIA,int64,0,13730,2,0.000000
IN_DISLEXIA,int64,0,13730,2,0.000000
IN_GESTANTE,int64,0,13730,2,0.000000


In [11]:
#Dataframe auxiliar na análise
cria_df_aux(test[features])

,Type,Missing,Size,Unique,Missing_%
CO_PROVA_CH,object,0,4576,9,0.000000
CO_PROVA_CN,object,0,4576,9,0.000000
CO_PROVA_LC,object,0,4576,9,0.000000
CO_PROVA_MT,object,0,4576,9,0.000000
CO_UF_RESIDENCIA,int64,0,4576,27,0.000000
IN_BAIXA_VISAO,int64,0,4576,2,0.000000
IN_CEGUEIRA,int64,0,4576,1,0.000000
IN_DISCALCULIA,int64,0,4576,1,0.000000
IN_DISLEXIA,int64,0,4576,1,0.000000
IN_GESTANTE,int64,0,4576,2,0.000000


### Mais algumas observações feitas nas variáveis selecionadas

Nota-se que a coluna 'IN_CEGUEIRA', apesar de aparentar um dado importante sobre o aluno, no caso em tela indica que há só 1 item único em sua base, o que indica que não há alunos com cegueira no Enem 2016 e, portanto, de ser uma coluna removida.

In [14]:
features.remove('IN_CEGUEIRA')

### Tratamento dados faltantes

A maior parte de variáveis que possuem dados faltantes são notas de provas, com exceção de 3 variáveis (TP_DEPENDENCIA_ADM_ESC,
TP_ENSINO e Q027). Dessa forma, optei por substituir os dados faltantes por 0, por ser o correspondente ao valor recebido pelo aluno que não possui nota na prova.

In [15]:
missing_data = ('NU_NOTA_CH','NU_NOTA_CN', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
                'NU_NOTA_COMP5', 'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO', 'Q027')

for i in missing_data:
    train[i].fillna(0, inplace = True)

train['NU_NOTA_MT'].fillna(0, inplace = True)

In [16]:
#Revendo o dataframe auxiliar na análise
cria_df_aux(train[features])

,Type,Missing,Size,Unique,Missing_%
CO_PROVA_CH,object,0,13730,10,0.0
CO_PROVA_CN,object,0,13730,10,0.0
CO_PROVA_LC,object,0,13730,9,0.0
CO_PROVA_MT,object,0,13730,9,0.0
CO_UF_RESIDENCIA,int64,0,13730,27,0.0
IN_BAIXA_VISAO,int64,0,13730,2,0.0
IN_DISCALCULIA,int64,0,13730,2,0.0
IN_DISLEXIA,int64,0,13730,2,0.0
IN_GESTANTE,int64,0,13730,2,0.0
IN_IDOSO,int64,0,13730,2,0.0


Farei o mesmo tratamento para a base test.

In [17]:
for i in missing_data:
    test[i].fillna(0, inplace = True)

In [18]:
#Revendo dataframe auxiliar na análise
cria_df_aux(test[features])

,Type,Missing,Size,Unique,Missing_%
CO_PROVA_CH,object,0,4576,9,0.0
CO_PROVA_CN,object,0,4576,9,0.0
CO_PROVA_LC,object,0,4576,9,0.0
CO_PROVA_MT,object,0,4576,9,0.0
CO_UF_RESIDENCIA,int64,0,4576,27,0.0
IN_BAIXA_VISAO,int64,0,4576,2,0.0
IN_DISCALCULIA,int64,0,4576,1,0.0
IN_DISLEXIA,int64,0,4576,1,0.0
IN_GESTANTE,int64,0,4576,2,0.0
IN_IDOSO,int64,0,4576,1,0.0


## Continuando a análise das variáveis para o modelo

Utilizaremos o método Backward stepwise selection(BSS), procurando remover as colunas que contribuem menos para o modelo de acordo com algumas métricas.

### Correlação entre as colunas numéricas

Depois de analisarmos o que conhecemos sobre a base de dados para remover colunas, vamos analisar a correlação entre os dados e as notas de matemática. Para isso vamos separar colunas categóricas de numéricas, eis que existem métodos mais apropriados para análise de correlação entre variáveis apenas numéricas e variáveis numéricas e categóricas. Importante notar que, apesar de algumas variáveis estarem listadas como dados numéricos, elas são essencialmente categóricas e se comportam como tal, e desta forma foram colocadas dentre as categóricas.

In [19]:
numerical_columns =['NU_IDADE', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
                    'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'NU_NOTA_MT']

categorical_columns = set(features) - set(numerical_columns)
categorical_columns = sorted(categorical_columns)
print(categorical_columns)

['CO_PROVA_CH', 'CO_PROVA_CN', 'CO_PROVA_LC', 'CO_PROVA_MT', 'CO_UF_RESIDENCIA', 'IN_BAIXA_VISAO', 'IN_DISCALCULIA', 'IN_DISLEXIA', 'IN_GESTANTE', 'IN_IDOSO', 'IN_SABATISTA', 'IN_SURDEZ', 'IN_TREINEIRO', 'Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q027', 'Q047', 'TP_ANO_CONCLUIU', 'TP_COR_RACA', 'TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO', 'TP_ESCOLA', 'TP_LINGUA', 'TP_NACIONALIDADE', 'TP_PRESENCA_CH', 'TP_PRESENCA_CN', 'TP_PRESENCA_LC', 'TP_SEXO', 'TP_ST_CONCLUSAO']


In [20]:
#Analisando correlação entre as variáveis numéricas
train[numerical_columns].corr()

,NU_IDADE,NU_NOTA_CH,NU_NOTA_CN,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_LC,NU_NOTA_REDACAO,NU_NOTA_MT
NU_IDADE,1.000000,-0.235354,-0.246333,-0.250980,-0.252200,-0.246713,-0.251110,-0.212317,-0.237674,-0.253577,-0.240874
NU_NOTA_CH,-0.235354,1.000000,0.969323,0.877637,0.857895,0.850698,0.860898,0.719970,0.934704,0.871199,0.903608
NU_NOTA_CN,-0.246333,0.969323,1.000000,0.864529,0.845978,0.837947,0.848201,0.706057,0.920917,0.857810,0.907039
NU_NOTA_COMP1,-0.250980,0.877637,0.864529,1.000000,0.942819,0.938204,0.959026,0.788569,0.917661,0.968156,0.882746
NU_NOTA_COMP2,-0.252200,0.857895,0.845978,0.942819,1.000000,0.965976,0.951199,0.828269,0.891824,0.979367,0.862709
NU_NOTA_COMP3,-0.246713,0.850698,0.837947,0.938204,0.965976,1.000000,0.953317,0.833303,0.883205,0.979547,0.854946
NU_NOTA_COMP4,-0.251110,0.860898,0.848201,0.959026,0.951199,0.953317,1.000000,0.819962,0.896716,0.978794,0.866827
NU_NOTA_COMP5,-0.212317,0.719970,0.706057,0.788569,0.828269,0.833303,0.819962,1.000000,0.736902,0.885568,0.722275
NU_NOTA_LC,-0.237674,0.934704,0.920917,0.917661,0.891824,0.883205,0.896716,0.736902,1.000000,0.904795,0.943954
NU_NOTA_REDACAO,-0.253577,0.871199,0.857810,0.968156,0.979367,0.979547,0.978794,0.885568,0.904795,1.000000,0.875929


Tendo em vista que procuramos as melhores correlações com as notas de matemática, vamos retirar a variável NU_IDADE e mantermos as demais colunas, que possuem boa correlação.

In [21]:
features.remove('NU_IDADE')

### Relação entre as variáveis categóricas e a variável dependente (que no caso é numérica)

Já para análise da relação entre variáveis categóricas e variáveis quantitativas não há um consenso sobre a melhor forma de fazê-la, pois as correlações de Pearson e de Spearman, por exemplo, medem se duas variáveis estão se "movendo" juntas, e em qual grau de mudança. Não se pode aplicar essa lógica a relação de variáveis categóricas com variáveis numéricas, pois o valor de uma variável categórica convertida em número ser mais alto que o outro não significa que o valor dele aumentou.

Diante disso, vamos analisar outras métricas em relação a essas variáveis.

In [22]:
#Convertendo variáveis categóricas em formato numérico
obj_list = train[categorical_columns].select_dtypes(include = "object").columns
print (obj_list)

Index(['CO_PROVA_CH', 'CO_PROVA_CN', 'CO_PROVA_LC', 'CO_PROVA_MT', 'Q001',
       'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q027', 'Q047', 'TP_SEXO'],
      dtype='object')


In [23]:
label_enc = LabelEncoder()

for feature in obj_list:
    train[feature] = label_enc.fit_transform(train[feature].astype(str))

In [24]:
train[obj_list]

,CO_PROVA_CH,CO_PROVA_CN,CO_PROVA_LC,CO_PROVA_MT,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047,TP_SEXO
0,5,1,1,8,3,3,2,0,0,2,8,0,1
1,3,7,1,8,0,0,1,0,0,0,0,0,0
2,1,3,2,2,3,3,2,0,0,0,0,0,0
3,1,3,2,2,7,4,4,2,1,2,6,3,0
4,1,3,2,2,4,3,2,0,0,1,6,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,4,4,1,8,4,4,6,1,1,1,3,0,1
13726,3,7,0,7,0,0,2,0,1,1,9,0,0
13727,8,8,0,7,2,6,1,0,1,0,0,0,0
13728,4,4,5,6,1,3,6,1,1,2,5,0,1


In [25]:
#Analisando OLS de todos os dados categóricos
ols = sm.ols(formula = '''NU_NOTA_MT~CO_PROVA_CH + CO_PROVA_CN + CO_PROVA_LC + CO_PROVA_MT + CO_UF_RESIDENCIA + IN_BAIXA_VISAO
                + IN_DISCALCULIA + IN_DISLEXIA + IN_GESTANTE + IN_IDOSO + IN_SABATISTA + IN_SURDEZ + IN_TREINEIRO + Q001+ Q002
                + Q006 + Q024 + Q025 + Q026 + Q027 + Q047 + TP_ANO_CONCLUIU + TP_COR_RACA + TP_DEPENDENCIA_ADM_ESC + TP_ENSINO
                + TP_ESCOLA + TP_LINGUA + TP_NACIONALIDADE + TP_PRESENCA_CH + TP_PRESENCA_CN + TP_PRESENCA_LC + TP_SEXO
                + TP_ST_CONCLUSAO''', data = train).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             NU_NOTA_MT   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.883
Method:                 Least Squares   F-statistic:                     3230.
Date:                Mon, 08 Jun 2020   Prob (F-statistic):               0.00
Time:                        12:41:15   Log-Likelihood:                -79348.
No. Observations:               13730   AIC:                         1.588e+05
Df Residuals:                   13697   BIC:                         1.590e+05
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                -81.5120     14.728     -5.535      0.000    -110.380     -52.644
CO_PROVA_CH                1.0741      0.411      2.613      0.009       0.268       1.880
CO_PROVA_CN               -0.0577      0.287     -0.201      0.841      -0.620       0.505
CO_PROVA_LC                0.1990      0.560      0.355      0.722      -0.898       1.296
CO_PROVA_MT                2.8311      0.726      3.898      0.000       1.408       4.255
CO_UF_RESIDENCIA           0.0946      0.073      1.289      0.197      -0.049       0.238
IN_BAIXA_VISAO            -5.2447     19.075     -0.275      0.783     -42.635      32.145
IN_DISCALCULIA            28.8388     78.451      0.368      0.713    -124.937     182.614
IN_DISLEXIA             -101.4765     78.518     -1.292      0.196    -255.384      52.430
IN_GESTANTE                4.2087     21.782      0.193      0.847     -38.487      46.905
IN_IDOSO                 -45.7287     55.474     -0.824      0.410    -154.465      63.008
IN_SABATISTA             -13.6453      6.847     -1.993      0.046     -27.067      -0.223
IN_SURDEZ                -39.6249     35.130     -1.128      0.259    -108.484      29.234
IN_TREINEIRO              -0.2937      2.745     -0.107      0.915      -5.674       5.087
Q001                       0.4891      0.395      1.239      0.215      -0.285       1.263
Q002                       1.4749      0.463      3.188      0.001       0.568       2.382
Q006                       5.9391      0.323     18.403      0.000       5.307       6.572
Q024                       8.6671      1.234      7.024      0.000       6.248      11.086
Q025                       3.9153      1.655      2.365      0.018       0.671       7.160
Q026                      -2.0178      1.268     -1.591      0.112      -4.504       0.468
Q027                      -0.0601      0.344     -0.175      0.861      -0.734       0.614
Q047                      13.7923      0.791     17.446      0.000      12.243      15.342
TP_ANO_CONCLUIU           -0.1519      0.319     -0.476      0.634      -0.778       0.474
TP_COR_RACA               -2.5151      0.701     -3.585      0.000      -3.890      -1.140
TP_DEPENDENCIA_ADM_ESC   -29.2598      6.709     -4.361      0.000     -42.410     -16.110
TP_ENSINO                -10.5471      1.890     -5.581      0.000     -14.252      -6.843
TP_ESCOLA                 68.8348     13.470      5.110      0.000      42.432      95.237
TP_LINGUA                -10.9907      1.427     -7.703      0.000     -13.788      -8.194
TP_NACIONALIDADE          -8.4605      2.593     -3.262      0.001     -13.544      -3.377
TP_PRESENCA_CH             5.5394      2.452      2.259      0.024       0.734      10.345
TP_PRESENCA_CN             5.5394      2.452      2.259      0.024       0.734  

### Análise dos resultados OLS

As nossas variáveis categóricas apresentaram um valor interessante de R² e R² ajustado, o que a princípio nos leva a crer a relevância em manter essas variáveis no modelo. Além disso, como F-statistic está alto e p-value (Prob (F-statistic)) é 0, há evidência que há uma regressão linear dessas variáveis com a classe desejada.

Entretanto, analisando cada uma das variáveis em tela, vemos que há várias cujo t-test está bem baixo e o p-value está alto(superior a 0.05). Quanto maior o t-test e menor o p-value, maior o indicativo que devemos rejeitar a hipótese nula (que a variável não tem relação com a variável dependente). Assim, valores altos de p-value e baixos de t é um indicativo que é uma variável que não possui boa relação com a classe e que poderiam ser removidas do modelo preditivo. Vamos remover algumas por grupos de semelhança e vermos como a análise dos dados se comportam.

In [26]:
#Removendo as variáveis que se referem a cor (tipo) da prova
ols = sm.ols(formula = '''NU_NOTA_MT~ CO_PROVA_MT + CO_UF_RESIDENCIA + IN_BAIXA_VISAO
                + IN_DISCALCULIA + IN_DISLEXIA + IN_GESTANTE + IN_IDOSO + IN_SABATISTA + IN_SURDEZ + IN_TREINEIRO + Q001+ Q002
                + Q006 + Q024 + Q025 + Q026 + Q027 + Q047 + TP_ANO_CONCLUIU + TP_COR_RACA + TP_DEPENDENCIA_ADM_ESC + TP_ENSINO
                + TP_ESCOLA + TP_LINGUA + TP_NACIONALIDADE + TP_PRESENCA_CH + TP_PRESENCA_CN + TP_PRESENCA_LC + TP_SEXO
                + TP_ST_CONCLUSAO''', data = train).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             NU_NOTA_MT   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.883
Method:                 Least Squares   F-statistic:                     3563.
Date:                Mon, 08 Jun 2020   Prob (F-statistic):               0.00
Time:                        12:42:06   Log-Likelihood:                -79352.
No. Observations:               13730   AIC:                         1.588e+05
Df Residuals:                   13700   BIC:                         1.590e+05
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                -80.0778     14.517     -5.516      0.000    -108.533     -51.622
CO_PROVA_MT                2.6050      0.391      6.669      0.000       1.839       3.371
CO_UF_RESIDENCIA           0.0932      0.073      1.270      0.204      -0.051       0.237
IN_BAIXA_VISAO            -4.9206     19.075     -0.258      0.796     -42.310      32.469
IN_DISCALCULIA            31.7854     78.450      0.405      0.685    -121.988     185.559
IN_DISLEXIA             -103.9251     78.519     -1.324      0.186    -257.834      49.984
IN_GESTANTE                3.9790     21.785      0.183      0.855     -38.722      46.680
IN_IDOSO                 -46.8276     55.480     -0.844      0.399    -155.575      61.920
IN_SABATISTA             -13.6062      6.848     -1.987      0.047     -27.029      -0.183
IN_SURDEZ                -40.6001     35.125     -1.156      0.248    -109.450      28.250
IN_TREINEIRO              -0.2877      2.745     -0.105      0.917      -5.669       5.094
Q001                       0.5025      0.395      1.273      0.203      -0.271       1.276
Q002                       1.4718      0.463      3.182      0.001       0.565       2.379
Q006                       5.9387      0.323     18.400      0.000       5.306       6.571
Q024                       8.7369      1.234      7.081      0.000       6.319      11.155
Q025                       3.8838      1.655      2.346      0.019       0.639       7.128
Q026                      -1.9990      1.269     -1.576      0.115      -4.486       0.488
Q027                      -0.0556      0.344     -0.162      0.872      -0.729       0.618
Q047                      13.8284      0.790     17.493      0.000      12.279      15.378
TP_ANO_CONCLUIU           -0.1473      0.319     -0.461      0.645      -0.773       0.479
TP_COR_RACA               -2.5093      0.701     -3.578      0.000      -3.884      -1.135
TP_DEPENDENCIA_ADM_ESC   -29.3668      6.709     -4.377      0.000     -42.518     -16.215
TP_ENSINO                -10.5235      1.890     -5.568      0.000     -14.228      -6.819
TP_ESCOLA                 69.0300     13.471      5.124      0.000      42.624      95.436
TP_LINGUA                -10.9841      1.427     -7.697      0.000     -13.781      -8.187
TP_NACIONALIDADE          -8.4708      2.593     -3.266      0.001     -13.554      -3.387
TP_PRESENCA_CH             7.4581      2.339      3.189      0.001       2.874      12.042
TP_PRESENCA_CN             7.4581      2.339      3.189      0.001       2.874      12.042
TP_PRESENCA_LC           446.0527      4.818     92.572      0.000     436.608     455.498
TP_SEXO                   23.6121      1.398     16.887      0.000      20.871      26.353
TP_ST_CONCLUSAO           -6.9251      1.121     -6.176      0.000      -9.123  

Não houve melhora significativa em nossos dados gerais. Continuemos as remoções:

In [27]:
#Removendo as variáveis que se referem a presença nas outras provas e ano de conclusão do ensino médio
ols = sm.ols(formula = '''NU_NOTA_MT~ CO_PROVA_MT + CO_UF_RESIDENCIA + IN_BAIXA_VISAO
                + IN_DISCALCULIA + IN_DISLEXIA + IN_GESTANTE + IN_IDOSO + IN_SABATISTA + IN_SURDEZ + IN_TREINEIRO + Q001+ Q002
                + Q006 + Q024 + Q025 + Q026 + Q027 + Q047 + TP_COR_RACA + TP_DEPENDENCIA_ADM_ESC + TP_ENSINO
                + TP_ESCOLA + TP_LINGUA + TP_NACIONALIDADE + TP_PRESENCA_LC + TP_SEXO
                + TP_ST_CONCLUSAO''', data = train).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             NU_NOTA_MT   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.883
Method:                 Least Squares   F-statistic:                     3824.
Date:                Mon, 08 Jun 2020   Prob (F-statistic):               0.00
Time:                        12:42:14   Log-Likelihood:                -79357.
No. Observations:               13730   AIC:                         1.588e+05
Df Residuals:                   13702   BIC:                         1.590e+05
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                -80.0417     14.356     -5.575      0.000    -108.182     -51.902
CO_PROVA_MT                2.6193      0.391      6.704      0.000       1.853       3.385
CO_UF_RESIDENCIA           0.0932      0.073      1.273      0.203      -0.050       0.237
IN_BAIXA_VISAO            -5.3390     19.078     -0.280      0.780     -42.735      32.057
IN_DISCALCULIA            31.5590     78.473      0.402      0.688    -122.259     185.377
IN_DISLEXIA             -103.7754     78.543     -1.321      0.186    -257.731      50.180
IN_GESTANTE                4.4624     21.791      0.205      0.838     -38.251      47.176
IN_IDOSO                 -47.6022     55.493     -0.858      0.391    -156.376      61.172
IN_SABATISTA             -13.7231      6.850     -2.003      0.045     -27.150      -0.297
IN_SURDEZ                -40.6693     35.135     -1.158      0.247    -109.539      28.201
IN_TREINEIRO              -0.0992      2.744     -0.036      0.971      -5.477       5.278
Q001                       0.5042      0.395      1.278      0.201      -0.269       1.278
Q002                       1.5028      0.462      3.255      0.001       0.598       2.408
Q006                       5.9257      0.323     18.362      0.000       5.293       6.558
Q024                       8.7201      1.234      7.067      0.000       6.302      11.139
Q025                       3.9212      1.656      2.369      0.018       0.676       7.166
Q026                      -2.0694      1.257     -1.646      0.100      -4.534       0.395
Q027                      -0.0885      0.336     -0.263      0.792      -0.748       0.571
Q047                      13.8338      0.790     17.517      0.000      12.286      15.382
TP_COR_RACA               -2.5408      0.702     -3.622      0.000      -3.916      -1.166
TP_DEPENDENCIA_ADM_ESC   -29.3000      6.711     -4.366      0.000     -42.455     -16.145
TP_ENSINO                -10.3708      1.877     -5.526      0.000     -14.050      -6.692
TP_ESCOLA                 69.0667     13.475      5.126      0.000      42.654      95.479
TP_LINGUA                -10.9156      1.427     -7.652      0.000     -13.712      -8.119
TP_NACIONALIDADE          -8.3799      2.593     -3.232      0.001     -13.462      -3.298
TP_PRESENCA_LC           459.8666      2.179    211.032      0.000     455.595     464.138
TP_SEXO                   23.6810      1.398     16.936      0.000      20.940      26.422
TP_ST_CONCLUSAO           -6.6964      0.979     -6.838      0.000      -8.616      -4.777
==============================================================================
Omnibus:                     2322.183   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            49319.819
Skew:                    

In [28]:
#Removendo as variáveis que se referem a solicitação de prova especial
ols = sm.ols(formula = '''NU_NOTA_MT~ CO_PROVA_MT + CO_UF_RESIDENCIA + Q001+ Q002
                + Q006 + Q024 + Q025 + Q026 + Q027 + Q047 + TP_COR_RACA + TP_DEPENDENCIA_ADM_ESC + TP_ENSINO
                + TP_ESCOLA + TP_LINGUA + TP_NACIONALIDADE + TP_PRESENCA_LC + TP_SEXO
                + TP_ST_CONCLUSAO''', data = train).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             NU_NOTA_MT   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.883
Method:                 Least Squares   F-statistic:                     5434.
Date:                Mon, 08 Jun 2020   Prob (F-statistic):               0.00
Time:                        12:42:18   Log-Likelihood:                -79361.
No. Observations:               13730   AIC:                         1.588e+05
Df Residuals:                   13710   BIC:                         1.589e+05
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                -79.6841     14.334     -5.559      0.000    -107.781     -51.587
CO_PROVA_MT                2.6049      0.391      6.670      0.000       1.839       3.370
CO_UF_RESIDENCIA           0.0984      0.073      1.344      0.179      -0.045       0.242
Q001                       0.5100      0.395      1.293      0.196      -0.263       1.283
Q002                       1.5064      0.461      3.267      0.001       0.603       2.410
Q006                       5.9224      0.322     18.372      0.000       5.291       6.554
Q024                       8.6531      1.233      7.017      0.000       6.236      11.070
Q025                       4.0057      1.655      2.420      0.016       0.762       7.250
Q026                      -2.0520      1.251     -1.641      0.101      -4.504       0.400
Q027                      -0.0962      0.334     -0.288      0.773      -0.751       0.558
Q047                      13.7817      0.784     17.568      0.000      12.244      15.319
TP_COR_RACA               -2.5581      0.701     -3.648      0.000      -3.932      -1.184
TP_DEPENDENCIA_ADM_ESC   -28.9278      6.701     -4.317      0.000     -42.062     -15.793
TP_ENSINO                -10.5476      1.875     -5.626      0.000     -14.222      -6.873
TP_ESCOLA                 68.5603     13.439      5.102      0.000      42.218      94.902
TP_LINGUA                -10.9077      1.426     -7.650      0.000     -13.703      -8.113
TP_NACIONALIDADE          -8.4858      2.592     -3.274      0.001     -13.567      -3.405
TP_PRESENCA_LC           459.8835      2.173    211.646      0.000     455.624     464.143
TP_SEXO                   23.6704      1.396     16.956      0.000      20.934      26.407
TP_ST_CONCLUSAO           -6.6933      0.801     -8.359      0.000      -8.263      -5.124
==============================================================================
Omnibus:                     2322.889   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            49359.954
Skew:                          -0.113   Prob(JB):                         0.00
Kurtosis:                      12.286   Cond. No.                     1.02e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.02e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [29]:
#Removendo as variáveis que se referem ao estado de residência e questionário socio-econômico
ols = sm.ols(formula = '''NU_NOTA_MT~ CO_PROVA_MT + Q002
                + Q006 + Q024 + Q047 + TP_COR_RACA + TP_DEPENDENCIA_ADM_ESC + TP_ENSINO
                + TP_ESCOLA + TP_LINGUA + TP_NACIONALIDADE + TP_PRESENCA_LC + TP_SEXO
                + TP_ST_CONCLUSAO''', data = train).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             NU_NOTA_MT   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.883
Method:                 Least Squares   F-statistic:                     7368.
Date:                Mon, 08 Jun 2020   Prob (F-statistic):               0.00
Time:                        12:42:21   Log-Likelihood:                -79368.
No. Observations:               13730   AIC:                         1.588e+05
Df Residuals:                   13715   BIC:                         1.589e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                -78.9488     14.000     -5.639      0.000    -106.391     -51.506
CO_PROVA_MT                2.5956      0.391      6.644      0.000       1.830       3.361
Q002                       1.9152      0.403      4.757      0.000       1.126       2.704
Q006                       5.9951      0.318     18.865      0.000       5.372       6.618
Q024                      10.1508      1.109      9.150      0.000       7.976      12.325
Q047                      13.9455      0.779     17.892      0.000      12.418      15.473
TP_COR_RACA               -2.7248      0.689     -3.957      0.000      -4.075      -1.375
TP_DEPENDENCIA_ADM_ESC   -29.0465      6.701     -4.335      0.000     -42.181     -15.912
TP_ENSINO                -10.6340      1.872     -5.680      0.000     -14.304      -6.964
TP_ESCOLA                 69.6422     13.430      5.186      0.000      43.318      95.967
TP_LINGUA                -11.2362      1.423     -7.895      0.000     -14.026      -8.447
TP_NACIONALIDADE          -8.4952      2.593     -3.277      0.001     -13.577      -3.413
TP_PRESENCA_LC           460.3926      2.157    213.462      0.000     456.165     464.620
TP_SEXO                   23.3964      1.389     16.848      0.000      20.674      26.118
TP_ST_CONCLUSAO           -6.1907      0.774     -8.000      0.000      -7.708      -4.674
==============================================================================
Omnibus:                     2311.395   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            48777.350
Skew:                          -0.106   Prob(JB):                         0.00
Kurtosis:                      12.231   Cond. No.                         238.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Análise

Depois das sucessivas remoções, notamos uma melhora na F-Statistic e também no condition number, o que pode indicar que as variáveis restantes podem ser uma boa fonte para predição da variável dependente. 

Entretanto, tendo em vista que a correlação da maioria das variáveis numéricas com a variável dependente foram bem fortes, vamos fazer mais umas análises de regresssão juntamente com as variáveis numéricas.

In [30]:
#Análise com as variáveis numéricas e categóricas restantes
ols = sm.ols(formula = '''NU_NOTA_MT~ CO_PROVA_MT + Q002 + Q006 + Q024 + Q047 + TP_COR_RACA + TP_DEPENDENCIA_ADM_ESC + TP_ENSINO
                + TP_ESCOLA + TP_LINGUA + TP_NACIONALIDADE + TP_PRESENCA_LC + TP_SEXO + TP_ST_CONCLUSAO + NU_NOTA_CH
                + NU_NOTA_CN + NU_NOTA_COMP1 + NU_NOTA_COMP2 + NU_NOTA_COMP3 + NU_NOTA_COMP4 + NU_NOTA_COMP5 + NU_NOTA_LC
                + NU_NOTA_REDACAO''', data = train).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             NU_NOTA_MT   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     6453.
Date:                Mon, 08 Jun 2020   Prob (F-statistic):               0.00
Time:                        12:42:36   Log-Likelihood:                -77397.
No. Observations:               13730   AIC:                         1.548e+05
Df Residuals:                   13707   BIC:                         1.550e+05
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                -54.3321     12.150     -4.472      0.000     -78.147     -30.517
CO_PROVA_MT                2.0595      0.339      6.079      0.000       1.395       2.724
Q002                       1.0120      0.349      2.898      0.004       0.327       1.697
Q006                       3.4813      0.278     12.522      0.000       2.936       4.026
Q024                       3.5674      0.967      3.690      0.000       1.672       5.463
Q047                       6.8146      0.686      9.937      0.000       5.470       8.159
TP_COR_RACA               -1.1816      0.598     -1.977      0.048      -2.353      -0.010
TP_DEPENDENCIA_ADM_ESC   -14.5124      5.812     -2.497      0.013     -25.905      -3.120
TP_ENSINO                 -4.6975      1.626     -2.890      0.004      -7.884      -1.511
TP_ESCOLA                 33.8660     11.654      2.906      0.004      11.022      56.710
TP_LINGUA                 -2.0569      1.242     -1.656      0.098      -4.491       0.378
TP_NACIONALIDADE          -4.1284      2.248     -1.837      0.066      -8.534       0.277
TP_PRESENCA_LC           123.5193      5.836     21.164      0.000     112.079     134.959
TP_SEXO                   23.4349      1.220     19.210      0.000      21.044      25.826
TP_ST_CONCLUSAO           -1.9346      0.674     -2.869      0.004      -3.256      -0.613
NU_NOTA_CH                -0.1079      0.011     -9.676      0.000      -0.130      -0.086
NU_NOTA_CN                 0.2637      0.012     22.877      0.000       0.241       0.286
NU_NOTA_COMP1             -0.0491      0.040     -1.215      0.224      -0.128       0.030
NU_NOTA_COMP2             -0.0414      0.045     -0.930      0.352      -0.129       0.046
NU_NOTA_COMP3             -0.0010      0.046     -0.022      0.982      -0.091       0.089
NU_NOTA_COMP4              0.0808      0.044      1.837      0.066      -0.005       0.167
NU_NOTA_COMP5              0.0706      0.022      3.239      0.001       0.028       0.113
NU_NOTA_LC                 0.4777      0.012     39.710      0.000       0.454       0.501
NU_NOTA_REDACAO            0.0598      0.005     13.082      0.000       0.051       0.069
==============================================================================
Omnibus:                      735.435   Durbin-Watson:                   1.959
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1766.988
Skew:                           0.326   Prob(JB):                         0.00
Kurtosis:                       4.632   Cond. No.                     3.64e+15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.72e-22. This mig

Seguindo a mesma lógica que trabalhamos nas variáveis categóricas, vamos trabalhar com todas as variáveis para ver se é possível melhorar o modelo

In [31]:
#Remoção de algumas variáveis categóricas com pior desempenho nessa análise
ols = sm.ols(formula = '''NU_NOTA_MT~ CO_PROVA_MT + Q002 + Q006 + Q024 + Q047 + TP_DEPENDENCIA_ADM_ESC + TP_ENSINO
                + TP_ESCOLA + TP_PRESENCA_LC + TP_SEXO + TP_ST_CONCLUSAO + NU_NOTA_CH
                + NU_NOTA_CN + NU_NOTA_COMP1 + NU_NOTA_COMP2 + NU_NOTA_COMP3 + NU_NOTA_COMP4 + NU_NOTA_COMP5 + NU_NOTA_LC
                + NU_NOTA_REDACAO''', data = train).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             NU_NOTA_MT   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     7467.
Date:                Mon, 08 Jun 2020   Prob (F-statistic):               0.00
Time:                        12:42:39   Log-Likelihood:                -77402.
No. Observations:               13730   AIC:                         1.548e+05
Df Residuals:                   13710   BIC:                         1.550e+05
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                -63.5092     11.782     -5.390      0.000     -86.604     -40.415
CO_PROVA_MT                2.0534      0.339      6.059      0.000       1.389       2.718
Q002                       1.0479      0.349      3.003      0.003       0.364       1.732
Q006                       3.5915      0.275     13.060      0.000       3.052       4.131
Q024                       3.8462      0.962      4.000      0.000       1.961       5.731
Q047                       6.9081      0.685     10.081      0.000       5.565       8.251
TP_DEPENDENCIA_ADM_ESC   -14.6112      5.813     -2.513      0.012     -26.006      -3.216
TP_ENSINO                 -4.7268      1.626     -2.908      0.004      -7.914      -1.540
TP_ESCOLA                 34.0855     11.657      2.924      0.003      11.236      56.935
TP_PRESENCA_LC           121.6608      5.798     20.984      0.000     110.296     133.025
TP_SEXO                   23.3395      1.216     19.192      0.000      20.956      25.723
TP_ST_CONCLUSAO           -1.8650      0.674     -2.767      0.006      -3.186      -0.544
NU_NOTA_CH                -0.1077      0.011     -9.654      0.000      -0.130      -0.086
NU_NOTA_CN                 0.2639      0.012     22.887      0.000       0.241       0.286
NU_NOTA_COMP1             -0.0473      0.040     -1.172      0.241      -0.127       0.032
NU_NOTA_COMP2             -0.0423      0.045     -0.950      0.342      -0.130       0.045
NU_NOTA_COMP3             -0.0029      0.046     -0.062      0.950      -0.093       0.087
NU_NOTA_COMP4              0.0815      0.044      1.854      0.064      -0.005       0.168
NU_NOTA_COMP5              0.0707      0.022      3.244      0.001       0.028       0.113
NU_NOTA_LC                 0.4812      0.012     40.227      0.000       0.458       0.505
NU_NOTA_REDACAO            0.0597      0.005     13.065      0.000       0.051       0.069
==============================================================================
Omnibus:                      734.213   Durbin-Watson:                   1.960
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1750.848
Skew:                           0.328   Prob(JB):                         0.00
Kurtosis:                       4.622   Cond. No.                     3.65e+15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.66e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [32]:
#Remoção de algumas variáveis numéricas com pior desempenho nessa análise
ols = sm.ols(formula = '''NU_NOTA_MT~ CO_PROVA_MT + Q002 + Q006 + Q024 + Q047 + TP_DEPENDENCIA_ADM_ESC + TP_ENSINO
                + TP_ESCOLA + TP_PRESENCA_LC + TP_SEXO + TP_ST_CONCLUSAO + NU_NOTA_CH + NU_NOTA_CN + NU_NOTA_COMP5 + NU_NOTA_LC
                + NU_NOTA_REDACAO''', data = train).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             NU_NOTA_MT   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     8866.
Date:                Mon, 08 Jun 2020   Prob (F-statistic):               0.00
Time:                        12:42:41   Log-Likelihood:                -77404.
No. Observations:               13730   AIC:                         1.548e+05
Df Residuals:                   13713   BIC:                         1.550e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                -63.5572     11.782     -5.395      0.000     -86.651     -40.464
CO_PROVA_MT                2.0403      0.339      6.022      0.000       1.376       2.704
Q002                       1.0541      0.349      3.021      0.003       0.370       1.738
Q006                       3.6027      0.275     13.104      0.000       3.064       4.142
Q024                       3.8463      0.962      4.000      0.000       1.961       5.731
Q047                       6.8918      0.685     10.060      0.000       5.549       8.235
TP_DEPENDENCIA_ADM_ESC   -14.6401      5.814     -2.518      0.012     -26.035      -3.245
TP_ENSINO                 -4.7538      1.625     -2.925      0.003      -7.940      -1.568
TP_ESCOLA                 34.1486     11.657      2.929      0.003      11.299      56.998
TP_PRESENCA_LC           120.6836      5.717     21.111      0.000     109.478     131.889
TP_SEXO                   23.3139      1.214     19.201      0.000      20.934      25.694
TP_ST_CONCLUSAO           -1.8779      0.674     -2.786      0.005      -3.199      -0.557
NU_NOTA_CH                -0.1078      0.011     -9.660      0.000      -0.130      -0.086
NU_NOTA_CN                 0.2641      0.012     22.909      0.000       0.241       0.287
NU_NOTA_COMP5              0.0739      0.027      2.718      0.007       0.021       0.127
NU_NOTA_LC                 0.4816      0.012     40.281      0.000       0.458       0.505
NU_NOTA_REDACAO            0.0580      0.008      7.207      0.000       0.042       0.074
==============================================================================
Omnibus:                      733.424   Durbin-Watson:                   1.960
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1739.269
Skew:                           0.330   Prob(JB):                         0.00
Kurtosis:                       4.614   Cond. No.                     2.70e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.7e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Análise

Depois das sucessivas remoções, notamos uma melhora na F-Statistic, o que pode indicar que as variáveis restantes podem ser uma boa fonte para predição da variável dependente.

In [37]:
#Removendo o restante das variáveis com baixo desempenho

to_stay = ['CO_PROVA_MT', 'Q002', 'Q006', 'Q024', 'Q047', 'TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO', 'TP_ESCOLA', 'TP_PRESENCA_LC',
           'TP_SEXO', 'TP_ST_CONCLUSAO', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_COMP5', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']

In [38]:
for elem in features:
    if elem in to_stay:
        print(elem)
    else:
        features.remove(elem)

CO_PROVA_MT
NU_NOTA_CH
NU_NOTA_CN
NU_NOTA_LC
NU_NOTA_REDACAO
Q002
Q006
Q024
Q047
TP_DEPENDENCIA_ADM_ESC
TP_ENSINO
TP_ESCOLA
TP_SEXO
TP_ST_CONCLUSAO


In [39]:
print(features)

['CO_PROVA_MT', 'IN_SURDEZ', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_COMP5', 'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'Q002', 'Q006', 'Q024', 'Q047', 'TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO', 'TP_ESCOLA', 'TP_PRESENCA_LC', 'TP_SEXO', 'TP_ST_CONCLUSAO']


### Análise de performance das regressões

Vamos analisar com base na validação cruzada o __R²__, __Mean Squared Error__ e __Median Absolute Error__ a performance da Regressão Linear, do Ridge e do Lasso no caso proposto. 

In [40]:
#Definir X e y do modelo
X = train[features].values
y = train['NU_NOTA_MT'].values

#Escalar as variáveis
scaler = StandardScaler()
X_scaled = scaler.fit_transform(train[features])

#Scores Regressão Linear
lin_reg = LinearRegression(fit_intercept=False)

scores = cross_validate(lin_reg, X, y, cv=3,
                        scoring=('r2', 'neg_mean_squared_error', 'neg_median_absolute_error'),
                        return_train_score=False)
print(np.mean(scores['test_r2']))
print(np.mean(scores['test_neg_mean_squared_error']))
print(np.mean(scores['test_neg_median_absolute_error']))

0.911277827931401
-4645.09996820323
-35.42023823333377


In [41]:
#Scores Lasso
lasso = Lasso()

scores_lasso = cross_validate(lasso, X_scaled, y, cv=3,
                        scoring=('r2', 'neg_mean_squared_error', 'neg_median_absolute_error'),
                        return_train_score=False)
print(np.mean(scores_lasso['test_r2']))
print(np.mean(scores_lasso['test_neg_mean_squared_error']))
print(np.mean(scores_lasso['test_neg_median_absolute_error']))

0.9106789956668986
-4676.4035988264395
-35.39820985552208


In [42]:
#Scores Ridge
ridge = Ridge()

scores_ridge = cross_validate(ridge, X_scaled, y, cv=3,
                        scoring=('r2', 'neg_mean_squared_error', 'neg_median_absolute_error'),
                        return_train_score=False)
print(np.mean(scores_ridge['test_r2']))
print(np.mean(scores_ridge['test_neg_mean_squared_error']))
print(np.mean(scores_ridge['test_neg_median_absolute_error']))

0.9114139516026804
-4637.9864077178545
-35.390927710549306


## Criação do modelo

Os dados ficaram muito próximos. Aparentemente o Ridge performou melhor na validação cruzada, apresentando o score de R² melhor.

In [43]:
ridge.fit(X, y)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [44]:
#Convertendo variáveis categóricas em formato numérico no dataframe test
obj_list_test = test[features].select_dtypes(include = "object").columns

for feature in obj_list_test:
    test[feature] = label_enc.fit_transform(test[feature].astype(str))

In [45]:
#Pegando os resultados
x_test = test[features].values
result = []
result = ridge.predict(x_test)

In [46]:
#Pegando o número de inscrição
Inscricao = test.loc[:, 'NU_INSCRICAO'].values
Inscricao = Inscricao.reshape(-1, 1)

In [47]:
#Montando o arquivo csv de resposta
df1 = pd.DataFrame(data=result, columns=["NU_NOTA_MT"])  
df2 = pd.DataFrame(data=Inscricao, columns=["NU_INSCRICAO"])

answer = pd.concat([df2, df1], axis = 1)

answer.to_csv('answer.csv', index = False)